# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

In [3]:
import os

# Read API key from text file
with open(r"D:\desktop\Pankaj_New_OpenAI_Key.txt", "r") as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()


Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

## Create our QandA application

In [7]:
from openai import OpenAI
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [8]:
file = r"D:\desktop\Lang_Chain\OutdoorClothingCatalog_fixed.csv"

loader = CSVLoader(file_path=file,encoding="utf-8")
data = loader.load()
print(data[0])

page_content='content: [Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}),' metadata={'source': 'D:\\desktop\\Lang_Chain\\OutdoorClothingCatalog_fixed.csv', 'row': 0}


In [9]:
len(data)

1000

In [13]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = DocArrayInMemorySearch.from_documents(data,embeddings)

In [14]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
llm = ChatOpenAI(temperature=0,model='gpt-3.5-turbo')

### Coming up with test datapoints

In [17]:
data[10]

Document(metadata={'source': 'D:\\desktop\\Lang_Chain\\OutdoorClothingCatalog_fixed.csv', 'row': 10}, page_content='content: Document(page_content=": 10\\nname: Cozy Comfort Pullover Set, Stripe\\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that\'s as comfortable at bedtime as it is when we have to make a quick run out.\\n\\nSize & Fit\\n- Pants are Favorite Fit: Sits lower on the waist.\\n- Relaxed Fit: Our most generous fit sits farthest from the body.\\n\\nFabric & Care\\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\\n\\nAdditional Features\\n- Relaxed fit top with raglan sleeves and rounded hem.\\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\\n\\nImported.", metadata={\'source\': \'OutdoorClothingCatalog_1000.csv\', \'row\': 10}),')

In [20]:
data[11]

Document(metadata={'source': 'D:\\desktop\\Lang_Chain\\OutdoorClothingCatalog_fixed.csv', 'row': 11}, page_content="content: Document(page_content=': 11\\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11}),")

### Hard-coded examples

In [23]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [26]:
from langchain_classic.evaluation.qa import QAGenerateChain

In [28]:
example_gen_chain = QAGenerateChain.from_llm(llm)

In [30]:
# the warning below can be safely ignored

In [32]:
new_examples = example_gen_chain.batch(
    [{"doc": t} for t in data[:5]]
)

In [34]:
new_examples[0]

{'doc': Document(metadata={'source': 'D:\\desktop\\Lang_Chain\\OutdoorClothingCatalog_fixed.csv', 'row': 0}, page_content='content: [Document(page_content=": 0\\nname: Women\'s Campside Oxfords\\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \\n\\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \\n\\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \\n\\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \\n\\nQuestions? Please contact us for any inquiries.", metadata={\'source\': \'OutdoorClothingCatalog_1

In [36]:
data[0]

Document(metadata={'source': 'D:\\desktop\\Lang_Chain\\OutdoorClothingCatalog_fixed.csv', 'row': 0}, page_content='content: [Document(page_content=": 0\\nname: Women\'s Campside Oxfords\\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \\n\\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \\n\\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \\n\\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \\n\\nQuestions? Please contact us for any inquiries.", metadata={\'source\': \'OutdoorClothingCatalog_1000.csv\

### Combine examples

In [39]:
examples += new_examples
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'doc': Document(metadata={'source': 'D:\\desktop\\Lang_Chain\\OutdoorClothingCatalog_fixed.csv', 'row': 0}, page_content='content: [Document(page_content=": 0\\nname: Women\'s Campside Oxfords\\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \\n\\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \\n\\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \\n\\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam m

In [57]:
print(examples[0])

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'answer': 'Yes'}


In [59]:
example_gen_chain.invoke(examples[0]["query"])

{'doc': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'qa_pairs': {'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
  'answer': 'The document does not provide information about whether the Cozy Comfort Pullover Set has side pockets.'}}

## Manual Evaluation

In [61]:
import langchain
langchain.debug = True

In [65]:
example_gen_chain.invoke(examples[0]["query"])

{'doc': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'qa_pairs': {'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
  'answer': 'The document does not provide information about whether the Cozy Comfort Pullover Set has side pockets or not.'}}

In [ ]:
# Turn off the debug mode
langchain.debug = False

## LLM assisted evaluation

In [ ]:
predictions = qa.apply(examples)

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [ ]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

In [ ]:
graded_outputs[0]

Reminder: Download your notebook to you local computer to save your work.